<a href="https://colab.research.google.com/github/Hushpuppyzac/DLI-Assignment/blob/main/eric_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

In [ ]:
import requests
import nbformat
from IPython import get_ipython

def run_notebook_from_github(url):
    """
    Downloads and executes a Jupyter notebook from a GitHub raw URL.

    Parameters:
    url (str): Raw GitHub URL to a .ipynb notebook file
    """
    print("🔄 Downloading notebook from GitHub...")
    try:
        response = requests.get(url)
        response.raise_for_status()
    except Exception as e:
        print(f"❌ Failed to download notebook: {e}")
        return

    print("📖 Parsing notebook content...")
    try:
        notebook = nbformat.reads(response.text, as_version=4)
    except Exception as e:
        print(f"❌ Failed to parse notebook: {e}")
        return

    ipython = get_ipython()
    print("⚙️  Running notebook cells...\n")

    for i, cell in enumerate(notebook.cells):
        if cell.cell_type == 'code':
            try:
                print(f"▶️  Executing cell [{i + 1}]...")
                ipython.run_cell(cell.source)
            except Exception as e:
                print(f"❌ Error in cell [{i + 1}]: {e}")

    print("\n✅ All executable cells have been processed.")

# 🔗 Your GitHub raw notebook URL
notebook_url = "https://raw.githubusercontent.com/Hushpuppyzac/DLI-Assignment/main/CleanedData.ipynb"

# ▶️ Run it
run_notebook_from_github(notebook_url)

🔄 Downloading notebook from GitHub...
📖 Parsing notebook content...
⚙️  Running notebook cells...

▶️  Executing cell [2]...
 AFTER CLEANING
Total rows after cleaning: 223108
Label distribution after cleaning:
Label
1    128016
0     95092
Name: count, dtype: int64
Class Distribution Before Balancing:
Label
1    128016
0     95092
Name: count, dtype: int64

 AFTER UNDERSAMPLING
Total rows after balancing: 190184
Class balance:
Label
1    95092
0    95092
Name: count, dtype: int64

SAMPLE OF BALANCED DATAFRAME
Total Rows     : 190184
DDoS Attacks   : 95092
Benign Records : 95092

 Balanced DataFrame (First 5 Rows):


,Destination Port,Flow Duration,Label
0,80,525915,1
1,53,173,0
2,80,33700,1
3,53,295753,0
4,443,117168001,0



 DDoS Samples (First 5):


,Destination Port,Flow Duration,Label
0,80,525915,1
2,80,33700,1
9,80,624202,1
10,80,2603947,1
11,80,70721898,1



 Benign Samples (First 5):


,Destination Port,Flow Duration,Label
1,53,173,0
3,53,295753,0
4,443,117168001,0
5,53,31435,0
6,80,11,0



 Sample Extracted Features (First 5):


,Flow Preview,pkt_length_diff,pkt_length_var_ratio,byte_ratio,duration_per_packet,avg_to_max_ratio
0,80 | 525915,5840,5.020,0.002,58434.935,0.221
1,53 | 173,16,1.194,0.729,43.250,1.047
2,80 | 33700,5840,5.020,0.002,3744.440,0.221
3,53 | 295753,103,2.091,0.539,49292.085,0.558
4,443 | 117168001,1430,7.575,0.147,4686718.165,0.137



 Final Columns:
['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd 

Algorithm : SVM
Confusion Matrix Heatmap

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 👉 Input features (you can change feature list if needed)
feature_cols = ['pkt_length_diff', 'pkt_length_var_ratio', 'byte_ratio', 'duration_per_packet', 'avg_to_max_ratio']
X = df_balanced[feature_cols]
y = df_balanced['Label']

# 👉 Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 👉 Normalize features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ✅ Train Calibrated SVM (for probability prediction)
svm_base = SVC(kernel='rbf', probability=True, random_state=42)
svm_clf = CalibratedClassifierCV(svm_base, cv=3)
svm_clf.fit(X_train_scaled, y_train)

# 🔍 Predict and evaluate
y_pred = svm_clf.predict(X_test_scaled)
y_proba = svm_clf.predict_proba(X_test_scaled)[:, 1]

# 📊 Metrics
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc = roc_auc_score(y_test, y_proba)

print("\n🎯 SVM Model Results:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")
print(f"ROC AUC  : {roc:.4f}")
